<a href="https://colab.research.google.com/github/mrzhbr/mrzhbr.github.io/blob/main/GoogleColabPytorch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import ssl
import matplotlib


ssl._create_default_https_context = ssl._create_unverified_context

In [3]:

kwargs = {}

train_data = torch.utils.data.DataLoader(
    datasets.EMNIST('data',split='byclass', train=True, download=True,
                    transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081,))])),
    batch_size=256, shuffle=True, **kwargs)



test_data = torch.utils.data.DataLoader(
    datasets.EMNIST('data',split='byclass', train=False,
                    transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((0.1307,),(0.3081,))])),
    batch_size=256, shuffle=True, **kwargs)




Extracting data/EMNIST/raw/emnist.zip to data/EMNIST/raw
Processing byclass


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!


In [30]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.dropout1 = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 128)
        self.fc2 = nn.Linear(128, 62)


    
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.dropout1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
        print(x.size())
        exit()





In [31]:
model = Netz()
model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.3)

def train(epoch):
    model.train()
    for batch_id, (data, target) in enumerate(train_data):
        data = data.cuda()
        target = target.cuda()
        data = Variable(data)
        target = Variable(target)
        optimizer.zero_grad()
        out = model(data)
        criterion = F.nll_loss
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format
            (epoch, batch_id*len(data), len(train_data.dataset),
                    100. * batch_id / len(train_data), loss.item()))



def test():
    model.eval()
    loss = 0
    correct = 0
    for data, target in test_data:
        data = Variable(data.cuda(), volatile = True)
        target = Variable(target.cuda())
        out  = model(data)
        loss += F.nll_loss(out, target, reduction='sum').item()
        prediction = out.data.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.data.view_as(prediction)).sum().item()
    loss = loss / len(test_data.dataset)
    print('Durchschnittsloss', loss)
    print('Accuracy: ', 100.*correct/len(test_data.dataset))


In [32]:
for epoch in range(1, 5):
    train(epoch)

    test()
test()
torch.save(model, 'emnist.pt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/697932 (0%)]	Loss: 4.131593
Train Epoch: 1 [256/697932 (0%)]	Loss: 4.095791
Train Epoch: 1 [512/697932 (0%)]	Loss: 4.059857
Train Epoch: 1 [768/697932 (0%)]	Loss: 3.991598
Train Epoch: 1 [1024/697932 (0%)]	Loss: 3.955657
Train Epoch: 1 [1280/697932 (0%)]	Loss: 3.858575
Train Epoch: 1 [1536/697932 (0%)]	Loss: 3.793379
Train Epoch: 1 [1792/697932 (0%)]	Loss: 3.755041
Train Epoch: 1 [2048/697932 (0%)]	Loss: 3.697265
Train Epoch: 1 [2304/697932 (0%)]	Loss: 3.706741
Train Epoch: 1 [2560/697932 (0%)]	Loss: 3.500344
Train Epoch: 1 [2816/697932 (0%)]	Loss: 3.538333
Train Epoch: 1 [3072/697932 (0%)]	Loss: 3.385697
Train Epoch: 1 [3328/697932 (0%)]	Loss: 3.629227
Train Epoch: 1 [3584/697932 (1%)]	Loss: 3.544466
Train Epoch: 1 [3840/697932 (1%)]	Loss: 3.439253
Train Epoch: 1 [4096/697932 (1%)]	Loss: 3.438185
Train Epoch: 1 [4352/697932 (1%)]	Loss: 3.341947
Train Epoch: 1 [4608/697932 (1%)]	Loss: 3.251241
Train Epoch: 1 [4864/697932 (1%)]	Loss: 3.264050
Train Epoch: 1 [5120/69793

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Streaming output truncated to the last 5000 lines.
Train Epoch: 3 [117760/697932 (17%)]	Loss: 0.483948
Train Epoch: 3 [118016/697932 (17%)]	Loss: 0.495317
Train Epoch: 3 [118272/697932 (17%)]	Loss: 0.621999
Train Epoch: 3 [118528/697932 (17%)]	Loss: 0.552249
Train Epoch: 3 [118784/697932 (17%)]	Loss: 0.592529
Train Epoch: 3 [119040/697932 (17%)]	Loss: 0.442346
Train Epoch: 3 [119296/697932 (17%)]	Loss: 0.561442
Train Epoch: 3 [119552/697932 (17%)]	Loss: 0.641441
Train Epoch: 3 [119808/697932 (17%)]	Loss: 0.514615
Train Epoch: 3 [120064/697932 (17%)]	Loss: 0.536604
Train Epoch: 3 [120320/697932 (17%)]	Loss: 0.634973
Train Epoch: 3 [120576/697932 (17%)]	Loss: 0.563964
Train Epoch: 3 [120832/697932 (17%)]	Loss: 0.479381
Train Epoch: 3 [121088/697932 (17%)]	Loss: 0.570508
Train Epoch: 3 [121344/697932 (17%)]	Loss: 0.585246
Train Epoch: 3 [121600/697932 (17%)]	Loss: 0.549003
Train Epoch: 3 [121856/697932 (17%)]	Loss: 0.673510
Train Epoch: 3 [122112/697932 (17%)]	Loss: 0.543363
Train Epoch: 

In [15]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [35]:
model = torch.load('emnist.pt')



In [37]:
test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Durchschnittsloss 0.42908779822565557
Accuracy:  84.4201060839215
